In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

import esmtools as et
from esmtools import stats

import shapefile
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gp
import regionmask
import matplotlib.patches as mpatches
import xesmf as xe

In [2]:
# Open dataset 
dr = '/home/jovyan/SOARS_2021/Data/EC-Earth3/pot_temp/'
fn = 'thetao_Omon_EC-Earth3_omip2_r1i1p1f1_gn_198001-201812.nc'
data_thetao = xr.open_dataset(dr+fn)
data_thetao

<xarray.Dataset>
Dimensions:         (bnds: 2, i: 362, j: 292, lev: 75, time: 468, vertices: 4)
Coordinates:
  * time            (time) object 1980-01-16 12:00:00 ... 2018-12-16 12:00:00
    longitude       (j, i) float64 ...
    latitude        (j, i) float64 ...
  * i               (i) int32 1 2 3 4 5 6 7 8 ... 356 357 358 359 360 361 362
  * j               (j) int32 1 2 3 4 5 6 7 8 ... 286 287 288 289 290 291 292
  * lev             (lev) float64 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds       (time, bnds) object ...
    longitude_bnds  (j, i, vertices) float64 ...
    latitude_bnds   (j, i, vertices) float64 ...
    lev_bnds        (lev, bnds) float64 ...
    thetao          (time, lev, j, i) float32 ...
Attributes: (12/51)
    CDI:                       Climate Data Interface version 1.9.8 (https://...
    history:                   Mon Jun 28 16:19:54 2021: ncrcat thetao_Omon_E...
    source:                    EC-Earth3 (2019): \naerosol: none\natmos: IFS ...
    institution:               AEMET, Spain; BSC, Spain; CNR-ISAC, Italy; DMI...
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               OMIP
    ...                        ...
    cmor_version:              3.4.0
    CDO:                       Climate Data Operators version 1.9.8 (https://...
    nominal_resolution:        100 km
    sub_experiment:            none
    sub_experiment_id:         none
    NCO:                       netCDF Operators version 4.9.9 (Homepage = htt...

In [3]:
# Change the time to be from DatetimeNoLeap to a Datetime64
data_thetao['time'] = np.arange('1980-01-01','2019-01-01',dtype = 'datetime64[M]')
data_thetao

<xarray.Dataset>
Dimensions:         (bnds: 2, i: 362, j: 292, lev: 75, time: 468, vertices: 4)
Coordinates:
  * time            (time) datetime64[ns] 1980-01-01 1980-02-01 ... 2018-12-01
    longitude       (j, i) float64 ...
    latitude        (j, i) float64 ...
  * i               (i) int32 1 2 3 4 5 6 7 8 ... 356 357 358 359 360 361 362
  * j               (j) int32 1 2 3 4 5 6 7 8 ... 286 287 288 289 290 291 292
  * lev             (lev) float64 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds       (time, bnds) object 1980-01-01 00:00:00 ... 2019-01-01 00...
    longitude_bnds  (j, i, vertices) float64 ...
    latitude_bnds   (j, i, vertices) float64 ...
    lev_bnds        (lev, bnds) float64 0.0 1.024 1.024 ... 5.8e+03 6.004e+03
    thetao          (time, lev, j, i) float32 ...
Attributes: (12/51)
    CDI:                       Climate Data Interface version 1.9.8 (https://...
    history:                   Mon Jun 28 16:19:54 2021: ncrcat thetao_Omon_E...
    source:                    EC-Earth3 (2019): \naerosol: none\natmos: IFS ...
    institution:               AEMET, Spain; BSC, Spain; CNR-ISAC, Italy; DMI...
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               OMIP
    ...                        ...
    cmor_version:              3.4.0
    CDO:                       Climate Data Operators version 1.9.8 (https://...
    nominal_resolution:        100 km
    sub_experiment:            none
    sub_experiment_id:         none
    NCO:                       netCDF Operators version 4.9.9 (Homepage = htt...

In [9]:
# select dimensions!
thetao = data_thetao['thetao']
thetao

<xarray.DataArray 'thetao' (time: 468, lev: 75, j: 292, i: 362)>
[3710210400 values with dtype=float32]
Coordinates:
  * time       (time) datetime64[ns] 1980-01-01 1980-02-01 ... 2018-12-01
    longitude  (j, i) float64 ...
    latitude   (j, i) float64 ...
  * i          (i) int32 1 2 3 4 5 6 7 8 9 ... 355 356 357 358 359 360 361 362
  * j          (j) int32 1 2 3 4 5 6 7 8 9 ... 285 286 287 288 289 290 291 292
  * lev        (lev) float64 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
Attributes:
    standard_name:  sea_water_potential_temperature
    long_name:      Sea Water Potential Temperature
    units:          degC
    comment:        Diagnostic should be contributed even for models using co...
    original_name:  thetao
    cell_methods:   area: mean where sea time: mean
    cell_measures:  area: areacello volume: volcello
    history:        2020-03-02T19:57:03Z altered by CMOR: Reordered dimension...

In [5]:
# Create the target grid (1x1 degree)
#ds_out = xe.util.grid_global(2, 2)
#ds_out  # contains lat/lon values of cell centers and boundaries.

# Create regridder for `bilinear` interpolation
#regridder = xe.Regridder(thetao, ds_out, 'bilinear', ignore_degenerate=True)
#### To reuse weights, you need to provide a filename or weights

# Perform the regridding
#dr_out = regridder(thetao)
#dr_out

In [6]:
#thetao = dr_out

In [7]:
blob = thetao.sel(lon=slice(210,225),lat=slice(40,50),time=slice('2013-07','2016-06'),lev=slice(0,500))
blob

ValueError: dimensions or multi-index levels ['lon', 'lat'] do not exist

In [ ]:
nonblob = thetao.sel(lon=slice(210,225),lat=slice(40,50),time=slice('1982-01','2010-12'),lev=slice(0,500))
nonblob

In [ ]:
fig, ax = plt.subplots()
ax.plot(blob.mean('time').mean('lat').mean('lon'),blob.lev)
ax.plot(nonblob.mean('time').mean('lat').mean('lon'),nonblob.lev)
ax.invert_yaxis()
ax.set_title('Vertical profile of potential temperature CESM2 ', fontsize=13, pad=10)
ax.set_xlabel('Temperature ($^{o}$ C)', fontsize=13)
ax.set_ylabel('Depth (m)', fontsize=13)
ax.grid('');

In [ ]:
diff = blob.mean('time') - nonblob.mean('time')
diff

In [ ]:
fig, ax = plt.subplots()
ax.plot(diff.mean('lat').mean('lon'),diff.lev)
ax.invert_yaxis()
ax.set_title('Vertical profile of potential temperature MRI-ESM2-0', fontsize=13, pad=10)
ax.set_xlabel('Temperature ($^{o}$ C)', fontsize=13)
ax.set_ylabel('Depth (m)', fontsize=13)
ax.grid('');

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,8))
fig.suptitle('Vertical profile of potential temperature MRI-ESM2-0')
    
ax1.plot(blob.mean('time').mean('lat').mean('lon'),blob.lev, '#F5B14C', label = 'Blob')
ax1.plot(nonblob.mean('time').mean('lat').mean('lon'),nonblob.lev, '#9D2EC5', label = 'Non-Blob')
ax1.set_title('Blob vs. Non-Blob')
ax1.set_xlabel('Temperature ($^{o}$ C)', fontsize=13)
ax1.set_ylabel('Depth (m)', fontsize=13)
ax1.invert_yaxis()
#ax1.set_facecolor('aliceblue')
ax1.legend()
ax1.grid(axis = 'y', linestyle = '--', linewidth = 0.5);

ax2.plot(diff.mean('lat').mean('lon'),diff.lev, '#47DBCD')
ax2.set_title('Difference of Blob and Non-Blob')
ax2.set_xlabel('Temperature ($^{o}$ C)', fontsize=13)
ax2.invert_yaxis()
ax2.grid(axis = 'y', linestyle = '--', linewidth = 0.5);
